# Lab 2

## Regressão Linear Múltipla

Vamos comparar os resultados de 3 regressões simples estimadas com os dados do banco *Advertisement*, uma para cada preditor e de uma regressão múltipla com os 3 preditores sendo controlados ao mesmo tempo.

In [0]:
import pandas as pd
import statsmodels.formula.api as sms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [0]:
# Lendo os dados da web
adv = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv")
adv = adv.drop(columns=["Unnamed: 0"], axis=1)  # Dropa a primeira coluna
adv.head()

### 3 Regressões Simples

In [0]:
model_tv = sms.ols(formula="sales ~ TV", data=adv).fit()
print(model_tv.summary())

In [0]:
model_radio = sms.ols(formula="sales ~ radio", data=adv).fit()
print(model_radio.summary())

In [0]:
model_news = sms.ols(formula="sales ~ newspaper", data=adv).fit()
print(model_news.summary())

### Adicionando os 3 preditores

In [0]:
# Vamos exibir os 3 parâmetros juntos só para facilitar a comparação
print("Modelo TV:\t" + str(model_tv.params[1]))
print("Modelo rádio:\t" + str(model_radio.params[1]))
print("Modelo News:\t" + str(model_news.params[1]))

In [0]:
# Agora, estimando com os 3 preditores ao mesmo tempo
model_complete  = sms.ols(formula="sales ~ TV + radio + newspaper",
                         data=adv).fit()
print(model_complete.summary())

In [0]:
# Verificando os resultados das 3 regressões numa só tabela
from statsmodels.iolib.summary2 import summary_col

In [0]:
summarytable = summary_col([model_tv, model_news, model_radio, model_complete], stars=True)
print(summarytable)

Perceba que o efeito da variável `newspaper` **desaparece**.  Isso acontece porque quando estimamos o efeito de newspaper controlando pelas outras variáveis, percebemos que o efeito de newspaper na verdade era relacionado à variável rádio devido ao fato das duas possuírem correlação. O efeito de newspaper estimado na regressão simples era, portanto, **efeito espúrio**. 

Vamos verificar a matriz de correlação dessas variáveis:

In [0]:
adv[["TV", "radio", "newspaper"]].corr()

Perceba que, enquanto os pares TV-Radio e TV-Newspaper tem uma correlação baixa, o par Radio-Newpaper tem uma correlação razoável (0,35).

Vamos também observar os intervalos de confiança dos estimadores:

In [0]:
model_complete.conf_int()

Agora, vamos usar nosso modelo para fazer predições.

In [0]:
yhat = model_complete.get_prediction()
print("Predictions:")
print(yhat.predicted_mean)

In [0]:
print("Confidence Intervals:")
print(yhat.conf_int())

# Exercício 1

### 1

Abra o banco de dados `Boston` e estime 3 regressões:

- A primeira prevendo `medv` usando como preditor `lstat`;
- A segunda prevendo `medv` usando como preditor `age`;
- A terceira usando ambos os preditores.

O que podemos dizer sobre o comportamento dos coeficientes estimados? E sobre o ajuste dos modelos?

### 2

Estime uma regressão usando todas as variáveis como preditoras. Todas são úteis para prever `medv`? Quais você deixaria e quais tiraria? Por quê?

Como foi o ajuste desse modelo?

Agora tire as variáveis que você considerou que não ajudam a estimação de `medv`. O que aconteceu com o ajuste do modelo?

In [0]:
# Lendo os dados
boston = pd.DataFrame.from_csv("https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv")
boston.head()

# Exercício 2

Vamos fazer algumas investigações no banco *credit*. Aí estão dados de cartão de crédito com as seguintes variáveis:

- **Income** - renda em milhares de dólares;
- **Limit** - limite de crédito;
- **Rating** - o *score* de crédito;
- **Card** - número de cartões de crédito que a pessoa possui;
- **Age** - idade;
- **Education** - anos de escolaridade;
- **Gender** - sexo;
- **Student** - se é estudante;
- **Married** - se é casado;
- **Ethnicity** - etnia e
- **Balance** - dívida média de cartão de crédito.

## 1

Estude a diferença entre o **balance** entre homens e mulheres usando um modelo de regressão. Interprete os resultados.

## 2
Estude a diferença entre o **balance** entre pessoas de diferentes etnias usando um modelo de regressão. Interprete os resultados.

## 3

Estime agora um modelo de regressão utilizando todas as variáveis presentes. O que podemos dizer sobre os resultados? Todas as variáveis ajudam a explicar o *balance*? Quais variáveis você tiraria? Como ficaria seu novo modelo?

## 4

Estime o seu novo modelo. Interprete os resultados. O que podemos dizer sobre o ajuste do novo modelo em relação ao primeiro?

In [0]:
# Lê os dados
credit = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Credit.csv")
# Tira a primeira coluna
print(credit.columns)
credit = credit.drop(columns=['Unnamed: 0'])
# Após retirar coluna com índices...
print(credit.columns)

In [0]:
credit.head()

In [0]:
model_fit = sms.ols(formula="Balance~Age+Rating+Limit", data=credit).fit()
print(model_fit.summary())

Verifica multicolinearidade com o *Variance Inflation Factor*.

In [0]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [0]:
#intercept = pd.DataFrame({"Intercept": [1] * credit.shape[0]})
new = credit[["Age", "Rating", "Limit"]]
new = add_constant(new)
colunas = new.columns
new = np.array(new)

In [0]:
vif = [variance_inflation_factor(new, i) for i in range(new.shape[1])]
res = zip(colunas, vif)
print(set(res))

## Extra

Replicando os gráficos de avaliação de resíduos do R

In [0]:
# Extraindo vetores necessários

# fitted values (need a constant term for intercept)
model_fitted_y = model_fit.fittedvalues

# model residuals
model_residuals = model_fit.resid

# normalized residuals
model_norm_residuals = model_fit.get_influence().resid_studentized_internal

# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))

# absolute residuals
model_abs_resid = np.abs(model_residuals)

# leverage, from statsmodels internals
model_leverage = model_fit.get_influence().hat_matrix_diag

# cook's distance, from statsmodels internals
model_cooks = model_fit.get_influence().cooks_distance[0]

In [0]:
import seaborn as sns
from statsmodels.graphics.gofplots import ProbPlot

In [0]:
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = sns.residplot(model_fitted_y, 'Balance', data=credit, 
                          lowess=True, 
                          scatter_kws={'alpha': 0.5}, 
                          line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')

# annotations
abs_resid = model_abs_resid.sort_values(ascending=False)
abs_resid_top_3 = abs_resid[:3]

for i in abs_resid_top_3.index:
    plot_lm_1.axes[0].annotate(i, 
                               xy=(model_fitted_y[i], 
                                   model_residuals[i]));

In [0]:
QQ = ProbPlot(model_norm_residuals)
plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)

plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0].set_title('Normal Q-Q')
plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
plot_lm_2.axes[0].set_ylabel('Standardized Residuals');

# annotations
abs_norm_resid = np.flip(np.argsort(np.abs(model_norm_residuals)), 0)
abs_norm_resid_top_3 = abs_norm_resid[:3]

for r, i in enumerate(abs_norm_resid_top_3):
    plot_lm_2.axes[0].annotate(i, 
                               xy=(np.flip(QQ.theoretical_quantiles, 0)[r],
                                  model_norm_residuals[i]));
plot_lm_2.show()

In [0]:
plot_lm_3 = plt.figure(3)
plot_lm_3.set_figheight(8)
plot_lm_3.set_figwidth(12)

plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5)
sns.regplot(model_fitted_y, model_norm_residuals_abs_sqrt, 
            scatter=False, 
            ci=False, 
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_3.axes[0].set_title('Scale-Location')
plot_lm_3.axes[0].set_xlabel('Fitted values')
plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$');

# annotations
abs_sq_norm_resid = np.flip(np.argsort(model_norm_residuals_abs_sqrt), 0)
abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]

for i in abs_norm_resid_top_3:
    plot_lm_3.axes[0].annotate(i, 
                               xy=(model_fitted_y[i], 
model_norm_residuals_abs_sqrt[i]));